## Data

In [15]:
import pandas as pd
import numpy as np
import gc
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import xgboost as xgb

# Load data files
hh_demand_12h_A = pd.read_csv('hh_demand_12h_A.csv')
hh_demand_true_A = pd.read_csv('hh_demand_true_A.csv')
Munich_weather_1min = pd.read_csv('Munich_weather_1min.csv')
Munich_weather_12h = pd.read_csv('Munich_weather_12h.csv')
dataA_params = pd.read_csv('DataA_params.csv', delimiter=';')

# Extract parameters
params = {row['Parameter']: float(row['Value']) for _, row in dataA_params.iterrows()}
params['c_M_PV'] = params.pop('c_MPV')  # Correct the parameter name

# Constants
alpha = params['alpha']
beta = params['beta']
gamma = params['gamma']
eta_B = params['eta_B']
eta_ref = params['eta_ref']
c_B = params['c_B']
c_M_PV = params['c_M_PV']
c_PV = params['c_PV']
G_ref = params['G_ref']
h = params['h']
i_INV = params['i_INV']
p_EL = params['p_EL']
p_FI = params['p_FI']
T_cref = params['T_cref']

# Prepare weather data
Munich_weather_1min['Timestamp'] = pd.to_datetime(Munich_weather_1min['Unnamed: 0'])
Munich_weather_1min.set_index('Timestamp', inplace=True)
Munich_weather_1min.drop(columns=['Unnamed: 0'], inplace=True)

time_range = pd.date_range(start='2013-01-01', end='2013-12-31 23:59:00', freq='T')
Munich_weather_12h['Timestamp'] = time_range[::720]  # Every 12 hours
Munich_weather_12h.set_index('Timestamp', inplace=True)
Munich_weather_12h.drop(columns=['Unnamed: 0'], inplace=True)

# Interpolation
Munich_weather_1min_full = Munich_weather_12h.reindex(time_range).interpolate(method='linear')

# Replace the 3 weeks with the actual 1-minute data provided
actual_data_periods = Munich_weather_1min.index

for period in actual_data_periods:
    Munich_weather_1min_full.loc[period] = Munich_weather_1min.loc[period]

# Free up memory
del Munich_weather_12h
gc.collect()

# Verify data lengths
print("Length of interpolated weather data:", len(Munich_weather_1min_full))
print("Length of household load data:", len(hh_demand_true_A))

# Add seasonal information
def get_season(date):
    if date.month in [12, 1, 2]:
        return 'winter'
    elif date.month in [3, 4, 5]:
        return 'spring'
    elif date.month in [6, 7, 8]:
        return 'summer'
    else:
        return 'autumn'

Munich_weather_1min_full['Season'] = Munich_weather_1min_full.index.to_series().apply(get_season)
Munich_weather_1min_full = pd.get_dummies(Munich_weather_1min_full, columns=['Season'])
Munich_weather_1min_full['Hour'] = Munich_weather_1min_full.index.hour
Munich_weather_1min_full['Day'] = Munich_weather_1min_full.index.dayofyear

# Prepare data for the ML model
X = Munich_weather_1min_full[['Hour', 'Day', 'Season_autumn', 'Season_spring', 'Season_summer', 'Season_winter']]
y_G = Munich_weather_1min_full['G_Gk']
y_T = Munich_weather_1min_full['Ta']

# Split the data into training and testing sets
X_train, X_test, y_G_train, y_G_test = train_test_split(X, y_G, test_size=0.2, random_state=42)
X_train_rf, X_test_rf, y_T_train, y_T_test = train_test_split(X, y_T, test_size=0.2, random_state=42)

# Train the Random Forest models
model_G_rf = RandomForestRegressor(n_estimators=100, random_state=42)
model_G_rf.fit(X_train, y_G_train)
model_T_rf = RandomForestRegressor(n_estimators=100, random_state=42)
model_T_rf.fit(X_train_rf, y_T_train)

# Get predictions from the Random Forest model
X_train['G_Gk_rf'] = model_G_rf.predict(X_train)
X_train['Ta_rf'] = model_T_rf.predict(X_train_rf)
X_test['G_Gk_rf'] = model_G_rf.predict(X_test)
X_test['Ta_rf'] = model_T_rf.predict(X_test_rf)

# Train the XGBoost models
model_G_xgb = xgb.XGBRegressor(n_estimators=100, random_state=42)
model_G_xgb.fit(X_train, y_G_train)
model_T_xgb = xgb.XGBRegressor(n_estimators=100, random_state=42)
model_T_xgb.fit(X_train, y_T_train)

# Predict the full year data with XGBoost
Munich_weather_1min_full['G_Gk_rf'] = model_G_rf.predict(X)
Munich_weather_1min_full['Ta_rf'] = model_T_rf.predict(X)
X['G_Gk_rf'] = Munich_weather_1min_full['G_Gk_rf']
X['Ta_rf'] = Munich_weather_1min_full['Ta_rf']
Munich_weather_1min_full['G_Gk_pred'] = model_G_xgb.predict(X)
Munich_weather_1min_full['Ta_pred'] = model_T_xgb.predict(X)

# Merge predicted weather data with household load data
merged_data = hh_demand_true_A.copy()
merged_data['G_Gk'] = Munich_weather_1min_full['G_Gk_pred'].values
merged_data['Ta'] = Munich_weather_1min_full['Ta_pred'].values


C:\Users\jihad\AppData\Local\Temp\ipykernel_23192\2769809030.py:40: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  time_range = pd.date_range(start='2013-01-01', end='2013-12-31 23:59:00', freq='T')


Length of interpolated weather data: 525600
Length of household load data: 525600


C:\Users\jihad\AppData\Local\Temp\ipykernel_23192\2769809030.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['G_Gk_rf'] = Munich_weather_1min_full['G_Gk_rf']
C:\Users\jihad\AppData\Local\Temp\ipykernel_23192\2769809030.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Ta_rf'] = Munich_weather_1min_full['Ta_rf']


## PV


In [16]:
import pyomo.environ as pyo
import numpy as np

# Create a Pyomo model
model = pyo.ConcreteModel()

# Ensure all values in G_Gk_pred and Ta_pred are standard Python floats
# Set negative values to zero
G_Gk_pred = {i+1: max(0, float(v)) for i, v in enumerate(Munich_weather_1min_full['G_Gk_pred'])}
Ta_pred = {i+1: float(v) for i, v in enumerate(Munich_weather_1min_full['Ta_pred'])}

# Ensure all parameter values are standard Python floats
params = {k: float(v) for k, v in params.items()}

# Define sets
model.T = pyo.RangeSet(1, len(G_Gk_pred))  # Time periods

# Define parameters
model.alpha = pyo.Param(initialize=params['alpha'])
model.beta = pyo.Param(initialize=params['beta'])
model.gamma = pyo.Param(initialize=params['gamma'])
model.eta_ref = pyo.Param(initialize=params['eta_ref'])
model.T_cref = pyo.Param(initialize=params['T_cref'])
model.G_ref = pyo.Param(initialize=params['G_ref'])
model.h = pyo.Param(initialize=params['h'])

# Define variables
model.CAP_PV = pyo.Var(within=pyo.NonNegativeReals, initialize=1)
model.CAP_B = pyo.Var(within=pyo.NonNegativeReals, initialize=1)
model.T_c = pyo.Var(model.T, within=pyo.Reals, initialize=0)
model.A_c = pyo.Var(within=pyo.NonNegativeReals, initialize=0)
model.G_t = pyo.Var(model.T, within=pyo.NonNegativeReals)
model.T_a_t = pyo.Var(model.T, within=pyo.Reals)

# Set values for G_t and T_a_t
for t in model.T:
    model.G_t[t] = G_Gk_pred[t]
    model.T_a_t[t] = Ta_pred[t]
    model.T_c[t] = model.T_a_t[t] + model.h * model.G_t[t]

# Define expressions
def eta_PV_expression(model, t):
    G_t_val = pyo.value(model.G_t[t])
    if G_t_val > 0:
        return model.eta_ref * (1 - model.beta * (model.T_c[t] - model.T_cref) + model.gamma * pyo.log(G_t_val / model.G_ref))
    else:
        return 0
model.eta_PV = pyo.Expression(model.T, rule=eta_PV_expression)

def P_PV_exp(model, t):
    model.A_c = pyo.value(model.alpha**(-1)) * pyo.value(model.CAP_PV)
    Ac=pyo.value(model.A_c)
    G_t=pyo.value(model.G_t[t])
    eta_Pv=pyo.value(model.eta_PV[t])
    return  Ac   * G_t  *eta_Pv / 1000
model.P_PV = pyo.Expression(model.T, rule=P_PV_exp)
# We do not define an objective function here because it will be defined in the NPV cell

## Battery

In [17]:
import pyomo.environ as pyo

# Assuming model is already created and necessary data is imported
model.eta_B = pyo.Param(initialize=params['eta_B'])


# Define additional variables and parameters for the battery model
model.P_B = pyo.Var(model.T, within=pyo.Reals, initialize=0)  # Battery power can be positive or negative
model.SOC = pyo.Var(model.T, within=pyo.NonNegativeReals, bounds=(0, 1), initialize=0)

# Initial state of charge
model.SOC_initial = pyo.Param(initialize=0)

# Define the household power demand P_HH based on the minute load
P_HH_dict = {i+1: v for i, v in enumerate(hh_demand_true_A['Load'])}
model.P_HH = pyo.Var(model.T, within=pyo.Reals)
for t in model.T:
    model.P_HH[t] = P_HH_dict[t]

# Define expressions for required power
def P_req_expression(model, t):
    return model.P_HH[t] - model.P_PV[t]
model.P_req = pyo.Expression(model.T, rule=P_req_expression)

# Define P_B expression
def P_B_exp(model, t):
    if t == 1:
        SOC_prev_value = model.SOC_initial
    else:
        SOC_prev_value = pyo.value(model.SOC[t-1])
    
    P_req_value = pyo.value(model.P_req[t])
    CAP_B_value = pyo.value(model.CAP_B)
    eta_B_value = pyo.value(model.eta_B)
    
    P_charge_limit = min(CAP_B_value * 60, (1 - SOC_prev_value) * CAP_B_value * 60 / eta_B_value)
    P_discharge_limit = min(CAP_B_value * 60, SOC_prev_value * eta_B_value * CAP_B_value * 60)
    
    # Ensure P_B is 0 when SOC is 1 and P_req is negative
    if SOC_prev_value == 1 and P_req_value < 0:
        P_B_value = 0
    elif P_req_value < 0:
        P_B_value = -min(P_charge_limit, -P_req_value)
    else:
        P_B_value = min(P_discharge_limit, P_req_value)
    
    # Update SOC
    if P_B_value > 0:
        SOC_value = SOC_prev_value - 1 / (eta_B_value * CAP_B_value * 60) * P_B_value
    else:
        SOC_value = SOC_prev_value - eta_B_value / (CAP_B_value * 60 ) * P_B_value
    
    # Ensure SOC is within bounds
    if SOC_value < 0:
        SOC_value = 0
    elif SOC_value > 1:
        SOC_value = 1
    
    model.SOC[t] = SOC_value  # Update SOC directly
    
    
    return P_B_value

model.P_B = pyo.Expression(model.T, rule=P_B_exp)

# Ensure SOC is within bounds
def SOC_bounds(model, t):
    return pyo.inequality(0, model.SOC[t], 1)
model.SOC_bounds = pyo.Constraint(model.T, rule=SOC_bounds)

'pyomo.core.base.var.IndexedVar'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.expression.IndexedExpression'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


## NPV

In [18]:
import pyomo.environ as pyo

# Define additional parameters for costs and rates
model.c_PV = pyo.Param(initialize=params['c_PV'])
model.c_B = pyo.Param(initialize=params['c_B'])
model.c_M_PV = pyo.Param(initialize=params['c_M_PV'])
model.p_EL = pyo.Param(initialize=params['p_EL'])
model.p_FI = pyo.Param(initialize=params['p_FI'])
model.i_INV = pyo.Param(initialize=params['i_INV'])

# Define expressions for power drawn from or fed to the grid
def P_D_expression(model, t):
    return model.P_HH[t] - model.P_PV[t] - model.P_B[t]
model.P_D = pyo.Expression(model.T, rule=P_D_expression)

# Define expressions for the aggregated quantities
def E_HH_T_expression(model):
    return sum(model.P_HH[t] for t in model.T)/60
model.E_HH_T = pyo.Expression(rule=E_HH_T_expression)

def E_D_plus_T_expression(model):
    return sum(model.P_D[t] for t in model.T if pyo.value(model.P_D[t]) > 0)/60
model.E_D_plus_T = pyo.Expression(rule=E_D_plus_T_expression)

def E_D_minus_T_expression(model):
    return sum(-model.P_D[t] for t in model.T if pyo.value(model.P_D[t]) < 0)/60
model.E_D_minus_T = pyo.Expression(rule=E_D_minus_T_expression)

# Define initial investments
def C_0_PV_expression(model):
    return model.c_PV * model.CAP_PV
model.C_0_PV = pyo.Expression(rule=C_0_PV_expression)

def C_0_B_expression(model):
    return model.c_B * model.CAP_B
model.C_0_B = pyo.Expression(rule=C_0_B_expression)

# Define annual revenue
def R_T_expression(model):
    return model.E_D_minus_T * model.p_FI + (model.E_HH_T - model.E_D_plus_T) * model.p_EL
model.R_T = pyo.Expression(rule=R_T_expression)

# Define NPV calculation
def NPV_expression(model):
    return -model.C_0_PV - model.C_0_B + sum((model.R_T - model.C_0_PV * model.c_M_PV) / (1 + model.i_INV)**t for t in range(1, 21))
model.NPV = pyo.Expression(rule=NPV_expression)

# Objective function to maximize NPV
model.objective = pyo.Objective(expr=model.NPV, sense=pyo.maximize)

In [19]:
print(pyo.value(model.NPV))

732.5171582367263


## Solver

In [ ]:
import pyomo.environ as pyo

# Configure solver options
solver = pyo.SolverFactory('ipopt')
solver.options['max_iter'] = 100  # Set maximum number of iterations
solver.options['print_level'] = 5  # Set the level of detail in the solver output

# Solve the model
results = solver.solve(model, tee=True)

# Extract and print optimal values
optimal_CAP_PV = pyo.value(model.CAP_PV)
optimal_CAP_B = pyo.value(model.CAP_B)
optimal_NPV = pyo.value(model.NPV)

print(f"Optimal Capacity for PV (CAP_PV) in kWp: {optimal_CAP_PV}")
print(f"Optimal Capacity for Battery (CAP_B) in kWh: {optimal_CAP_B}")
print(f"Optimal Net Present Value (NPV): {optimal_NPV}")

# Display detailed iteration results
print("\nDetailed Iteration Results:")
iteration_info = results.solver.termination_condition
print(iteration_info)

# Extracting and displaying more detailed iteration information
if hasattr(results.solver, 'iterations'):
    for iter_num, iter_data in enumerate(results.solver.iterations, start=1):
        print(f"Iteration {iter_num}: {iter_data}")

Ipopt 3.12.13: max_iter=100
print_level=5


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:   525600
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:  1051202
                     variables with only lower bounds:        2
                variables with lower and upper bounds:   525600
                     variables with only

## Test Model
